In [1]:
import re
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

def remove_repeated_sentences(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    unique_sentences = []
    for sentence in sentences:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
        else:
            break  # Stop adding sentences after the first repetition
    return ' '.join(unique_sentences)

def clean_response(response_text, prompt):
    # Remove the prompt from the response
    stripped_response = response_text.replace(prompt, '').strip()

    # Split the stripped response text into lines
    lines = stripped_response.split('\n')

    combined_lines = " ".join(line.strip() for line in lines if line.strip())
    return remove_repeated_sentences(combined_lines)

def generate_response(prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate response
    with torch.no_grad():
        output = model.generate(input_ids, max_length=100, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    cleaned_response = clean_response(response, prompt)

    return cleaned_response

In [2]:
print("AI: Hello! How can I help?")
while True:
    user_input = input("Human: ")
    if user_input.lower() == "exit":
        print("AI: Bye!")
        break

    response = generate_response(user_input)
    print("AI:", response)

AI: Hello! How can I help?


Human:  What does the string "Rust" refer to ?


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


AI: The string "Rust" refers to the Rust programming language.


Human:  What does the string " SolidGoldMagikarp" refer to ?


...

AI: [12:55:56]SAY: Ghost/JarekTheRaptor : I'm not sure if I can do it [12:55:57]SAY: Ghost/JarekTheRaptor : I'm not sure if I can do it [12:55:58]SAY: Ghost/JarekTheRaptor : I'm not sure if I


Human:  Can you repeat back the string ' SolidGoldMagikarp' to me please?


AI: I'm not sure if you can repeat back the string 'salt' to me please.


Human:  exit


AI: Bye!
